In [13]:
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join
print(pd.__version__)

0.22.0


In [14]:
# Combine Advanced and Regular statistics dataframes
regular = pd.read_csv('./data/computed/total_player_statistics.csv')
advanced = pd.read_csv('./data/computed/total_player_advanced_statistics.csv')

# Drop the column that index gets set to
regular.drop(inplace=True, columns="Unnamed: 0")
advanced.drop(inplace=True, columns="Unnamed: 0")

# We have already checked that every player in regular is in advanced and vice versa

# We want to merge entries in regular and advanced that contain the same player in the same year. Thus, we create a dummy column
# We drop duplicates for some players with the same names, since their contract data is not distinct
regular['player_year'] = regular[['Player','Year']].apply(lambda x: x['Player'] + str(int(x['Year'])) , axis=1)
advanced['player_year'] = advanced[['Player','Year']].apply(lambda x: x['Player'] + str(int(x['Year'])) , axis=1)
regular = regular.drop(columns=['Player','Year','Age'],axis=1)

all_stats = pd.merge(regular,advanced,on='player_year')
all_stats = all_stats.drop_duplicates(subset=['player_year'])

In [15]:
# Combine all player statistics with contract data
contracts = pd.read_csv('./data/computed/total_contract_data.csv')
contracts = contracts.drop('Unnamed: 0', axis=1)

players = contracts['Player'].unique()

contracts['player_year'] = contracts[['Player','Signed']].apply(lambda x: x['Player'] + str(int(x['Signed'])) , axis=1)
total_df = pd.DataFrame()

for i in range(len(contracts)):
    contract = contracts.iloc[i]
    player = contract['Player']
    year = contract['Signed']
    amt = contract['Dollars_Per_Year']
    if(player == 'LeBron James'):
        print(player)
        print(year)
        print(amt)
    player_stats = all_stats[all_stats['Player'] == player]
    for y in range(year-2,year+1):
        player_year = player_stats[player_stats['Year'] == y]
        if(len(player_year) == 1):
            player_year['Signed'] = year
            # Our output for a given year
            player_year['Dollars_Per_year'] = amt
            total_df = total_df.append(player_year)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


LeBron James
2010
18306250.0


In [21]:
total_df.to_csv(path_or_buf='./data/computed/training_data.csv')